In [1]:
import requests
import helpers

def sparqlQuery(query):
    url = 'https://query.wikidata.org/bigdata/namespace/wdq/sparql'
    return requests.get(url, params={'query': query, 'format': 'json'}).json()

def toDict(results):
    var = results['head']['vars']
    if (len(var) > 2):
        raise ValueError('More than 2 results vars')
    dic = {}
    for item in results['results']['bindings']:
        dic.update({ item[var[0]]['value'] : item[var[1]]['value'] })

    return dic




In [2]:
def retriveDepicts(painting=None):
    if painting == None:
        return None
    
    data = sparqlQuery("SELECT ?depicts ?depictsLabel WHERE { " \
            "    wd:" + painting + " wdt:P180 ?depicts." \
            "    SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }" \
            "}" \
            "LIMIT 100")

    return dict(map(lambda kv: (kv[0][kv[0].rindex("/")+1:], kv[1]), toDict(data).items()))

In [3]:
def retrieveSuperclass(obj=None):
    if obj == None:
        return None
    data = sparqlQuery("SELECT ?instance ?instanceLabel WHERE {" \
            "   wd:" + obj + " wdt:P279|wdt:P31 ?instance." \
            "   SERVICE wikibase:label { bd:serviceParam wikibase:language \"[AUTO_LANGUAGE],en\". }" \
            "}" \
            "LIMIT 100")
    return dict(map(lambda kv: (kv[0][kv[0].rindex("/")+1:], kv[1]), toDict(data).items()))

def retrieveSuperclassRecursively(obj=None, depth=1):
    if depth < 1 or obj==None:
        return None
    queue = [obj]
    res = {}
    for d in range(0,depth):
        new_objs = {}
        while len(queue) > 0:
            new_objs = new_objs | retrieveSuperclass(obj=queue.pop(0))
        queue = [*new_objs]
        res = res | new_objs
    return res



In [4]:
painting1 = 'Q1091086'

depicts = retriveDepicts(painting1) #The Third of May 1808

depicts

{'Q161936': 'Death',
 'Q181146': 'Dos de Mayo Uprising',
 'Q216169': 'execution by firing squad',
 'Q486396': 'shotgun'}

In [5]:
painting2 = 'Q5801323' 

depicts = retriveDepicts(painting2) #The Defense of Cadiz against the English

depicts

{'Q11446': 'ship',
 'Q12791': 'sword',
 'Q12796': 'firearm',
 'Q14660': 'flag',
 'Q44475': 'spear',
 'Q81103': 'cannon',
 'Q131559': 'shield',
 'Q181469': 'galleon',
 'Q1075310': 'sailboat',
 'Q2167774': 'Cádiz Expedition',
 'Q2207370': 'seat',
 'Q3579211': 'combat helmet',
 'Q50414774': 'armor'}

In [6]:
retrieveSuperclassRecursively(obj='Q486396', depth=1) #shotgun

{'Q12796': 'firearm',
 'Q15142889': 'weapon family',
 'Q1046481': 'long gun',
 'Q1064392': 'smoothbore'}

In [10]:
retrieveSuperclassRecursively(obj='Q44475', depth=3) #firearm

{'Q649107': 'pole weapon',
 'Q768186': 'sports equipment',
 'Q222405': 'close combat weapon',
 'Q2184715': 'durable good',
 'Q16798631': 'equipment',
 'Q125734': 'personal weapon',
 'Q28877': 'goods',
 'Q2424752': 'product',
 'Q8205328': 'artificial physical object'}